In [130]:
import requests
from pyvis.network import Network
import argparse
import os
import math
import json

In [75]:
# address="kaspa:qqssy8x2stwk6x7trmw56m8rwfkwul70rpqxrvv789mxqz73pdny2sprry82x"
address="kaspa:qp2sp0vvrwu4s8pw0j68muu2ta5qar5mehf8ehuvljw5zsrakk5cvx4gvqz7z"
nb_cercles=4
limit=100

In [119]:
URLADDRESS="https://api.kaspa.org/addresses/"
URLTRANSAC="https://api.kaspa.org/transactions/"

responseAddress=requests.get(URLADDRESS+address+"/full-transactions")
data=responseAddress.json()

In [102]:
print(data[9])

{'subnetwork_id': '0000000000000000000000000000000000000000', 'transaction_id': '7389b3b16ef5108920c6988b573df366048261d15aac19a9f0e7ca65415263b9', 'hash': '5fd776a9cee1736ebe4bd149bd8c554d850522214bf0331f73961851ef76cdfa', 'mass': '4272', 'payload': None, 'block_hash': ['350f85ecaf941599024219efb51cba996aa48ec8549616e1ad94323c875fb464'], 'block_time': 1715539318974, 'is_accepted': True, 'accepting_block_hash': 'b95a327c6e0b23d5b2e02fa91fe1157e1a1551ba18d87ca00a6fb7ba7026602f', 'accepting_block_blue_score': 77619366, 'accepting_block_time': 1715539320858, 'inputs': [{'transaction_id': '7389b3b16ef5108920c6988b573df366048261d15aac19a9f0e7ca65415263b9', 'index': 0, 'previous_outpoint_hash': 'f9d0979f0db2f15a1a313dcc82f8c14987b296743694e6c95fcdff70be4ab25c', 'previous_outpoint_index': '0', 'previous_outpoint_address': None, 'previous_outpoint_amount': None, 'signature_script': '4177f760b1a9d50b1c04faaade73773afae6703b5980c837db5bd2d4a77976390f0bdfc8de5d61944f80b3741fa727d56cb7af435c28eef3

In [139]:
def get_address_data(address):
    return requests.get(URLADDRESS+address+"/full-transactions").json()

def get_input(input):
    old_transac=input["previous_outpoint_hash"]
    old_index=int(input["previous_outpoint_index"])
    responseTransac=requests.get(URLTRANSAC+old_transac)
    dataTransac=responseTransac.json()
    for old_output in dataTransac["outputs"]:
        if old_output["index"]==old_index:
            return old_output["script_public_key_address"]
        
def get_inputs_ouputs(transac):
    inputs=[]
    outputs=[]

    for input in transac["inputs"]:
        inputs.append(get_input(input))

    for output in transac["outputs"]:
        output_add=output["script_public_key_address"]
        outputs.append(output_add)

    for output in outputs:
        if output in inputs:
            outputs.remove(output)
    return inputs,outputs

def explore_address(address):
    data_address=get_address_data(address)
    relations={}

    for transac in data_address:
        inputs,outputs=get_inputs_ouputs(transac)
        for input in inputs:
            if input not in relations:
                relations[input]={
                    "nb_transacs_in":0,
                    "nb_transacs_out":0,
                    "address_in":{},
                    "address_out":{}
                    }
            
            relations[input]["nb_transacs_out"]+=len(outputs)

            for output in outputs:
                if output in relations[input]["address_out"]:
                    relations[input]["address_out"][output]+=1
                else:
                    relations[input]["address_out"][output]=1

        for output in outputs:
            if output not in relations:
                relations[output]={
                    "nb_transacs_in":0,
                    "nb_transacs_out":0,
                    "address_in":{},
                    "address_out":{}
                }

            relations[output]["nb_transacs_in"]+=len(inputs)

            for input in inputs:
                if input in relations[output]["address_in"]:
                    relations[output]["address_in"][input]+=1
                else:
                    relations[output]["address_in"][input]=1

    return relations

def create_vis2(relations,initial_address):
    net=Network(
        height="900px", 
        width="100%", 
        bgcolor="#222222", 
        font_color="white",
        select_menu=True,
        filter_menu=True
    )

    net.barnes_hut(
        gravity=-10000, 
        central_gravity=0.3, 
        spring_length=200,
        spring_strength=0.01, 
        damping=0.09, 
        overlap=0
    )

    all_address=set()
    all_address.update(relations.keys())

    for input,outputs in relations.items():
        for output in outputs:
            if output!="nb_transacs":
                all_address.add(output)
    
    address_colors={}
    address_colors[initial_address]="#0000FF"
    colors=["#FF0000","#FFA500","#FFFF00","#800080","#00FF00","#00FFFF","#FF00FF"]
    index_color=0


    for address in all_address:
        if address==initial_address:
            continue
            
        else:
            address_colors[address]=colors[index_color]
    
def create_vis(relations, initial_address):
    net = Network(height="900px", width="100%", bgcolor="#222222", font_color="white", directed=True)
    net.barnes_hut(gravity=-10000, central_gravity=0.3, spring_length=250, spring_strength=0.01)

    for addr in relations.keys():
        if addr == initial_address:
            color = "#0000FF"
            size = 40
        else:
            color = "#FF0000"
            size = 25
        
        net.add_node(addr, label=addr[:10], title=addr, color=color, size=size)
    for source, data in relations.items():
        for target, count in data["address_out"].items():
            if target in relations:
                if source == initial_address:
                    edge_color = "#FF0000"
                    label=str(count)
                elif target == initial_address:
                    edge_color = "#FFFF00" 
                    label =str(count)
                else:
                    edge_color = "#666666"
                    label =str(count)

                net.add_edge(source, target, color=edge_color, value=count, title=f"{count} transacs", label=label)

    net.show("crypto_circles.html", notebook=False)


create_vis(explore_address(address),address)

crypto_circles.html


In [133]:
print(address)
relations=explore_address(address)
for add in relations:
    print(add,relations[add])

kaspa:qp2sp0vvrwu4s8pw0j68muu2ta5qar5mehf8ehuvljw5zsrakk5cvx4gvqz7z
kaspa:qp2sp0vvrwu4s8pw0j68muu2ta5qar5mehf8ehuvljw5zsrakk5cvx4gvqz7z {'nb_transacs': 13, 'kaspa:qpjfvypc2rlxzxwh3p578xt6ud663uv358mf20wjwqv9p832t2zjj8jps2mvd': 1, 'kaspa:qztrxqwp896ukrsn0yvd698hp8ww2dl0njjvts84cuymmh9hf9zg2mqh9j5mr': 3, 'kaspa:qrzt54pavj9gt0vgal7a2s79w3azvvpaqvpyftwf8ehxgju38p23ql965vkey': 6, 'kaspa:qpn3cnk57ymyhz6jc4wvleuquxc87dtyupzdeefq0lulwu0g8ryd22ww5cym7': 2, 'kaspa:qqxnx2ju886z9mkqj63m24ddcjs9f8h4w4wfhvmejje59g58pqnz6e2nh7mmg': 1}
kaspa:qzuft3vgkry6wqrqu693gxqrynvylzfu9e9p05lv5jflg72gw7qqqt0660jvc {'nb_transacs': 1, 'kaspa:qp2sp0vvrwu4s8pw0j68muu2ta5qar5mehf8ehuvljw5zsrakk5cvx4gvqz7z': 1}
kaspa:qzy5tjqc9fy0q66gy02tmrhngd5rwgkgj68nxpk9y67xgehy7kkljpjljst78 {'nb_transacs': 2, 'kaspa:qp2sp0vvrwu4s8pw0j68muu2ta5qar5mehf8ehuvljw5zsrakk5cvx4gvqz7z': 1, 'kaspa:qzuft3vgkry6wqrqu693gxqrynvylzfu9e9p05lv5jflg72gw7qqqt0660jvc': 1}
kaspa:qqxnx2ju886z9mkqj63m24ddcjs9f8h4w4wfhvmejje59g58pqnz6e2nh7mmg {'nb_trans